In [ ]:
# All Libraries required for this lab are listed below. The libraries pre-installed on Skills Network Labs are commented.
!pip install -qy pandas==1.3.4 numpy==1.21.4 seaborn==0.9.0 matplotlib==3.5.0 torch=2.1.0+cu118
# - Update a specific package
!pip install pmdarima -U
# - Update a package to specific version
!pip install --upgrade pmdarima==2.0.2


Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: -y
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 49.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 4.8 MB/s eta 0:00:00
  Attempting uninstall: pmdarima
    Found existing installation: pmdarima 2.0.4
    Uninstalling pmdarima-2.0.4:
      Successfully uninstalled pmdarima-2.0.4


In [ ]:
!pip install transformers==4.40.0
!pip install -U git+https://github.com/huggingface/transformers
!pip install datasets # 2.15.0
!pip install portalocker>=2/0.0
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install torch==2.3.0
!pip install -U torchvision
!pip install protobuf==3.20.*
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 80.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 128.3 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.1
    Uninstalling tokenizers-0.21.1:
      Successfully uninstalled tokenizers-0.21.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.51.3
    Uninstalling transformers-4.51.3:
      Successfully uninstalled transformers-4.51.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 3.4.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.40.0 which is incompatible.
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-mufs6uti
  Running command git clone --fi

In [ ]:
import torch
from torch.optim.lr_scheduler import LambdaLR
from torch.utils.data import DataLoader
from torch.optim import AdamW
from transformers import AutoConfig,AutoModelForCausalLM,AutoModelForSequenceClassification,BertConfig,BertForMaskedLM,TrainingArguments, Trainer, TrainingArguments
from transformers import AutoTokenizer,BertTokenizerFast,TextDataset,DataCollatorForLanguageModeling
from transformers import pipeline
# from datasets import load_dataset

from tqdm.auto import tqdm
import math
import time
import os


# You can also use this section to suppress warnings generated by your code:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

In [ ]:
# Set the environment variable TOKENIZERS_PARALLELISM to 'false'

import os
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

# Pretraining and self-supervised find-tuning

Pretraining is a technique used in natural language processing (NLP) to train large language model (LLMs) on a vast corpus of unlabeled text data.  The goal is to capture the general patterns and semantic relationships persent in natural language, allowing the model to develop a deep understanding of language structure and meaning.

The motivation gehind pretraining trainsformers is to address the limitations of traditional NLP approaches that often requires significant amounts of labeled data for each specific task.  Bynleveraging the abundance of unlabeled text data, pretraining enables the model to learn fundamental language skills through self-supervised objectives, facilitating transfer learning.

The pretraining objectives, such as masked language modeling (MLM) and next sentence prediction (NSP). play crucial role in the success of transformer models.  Pretrained models can be further tuned by training them on domain-specific unlabeled data, which is known as self-supervised fine-tuning.

Also, the model can be fine-tuned on specific downstream tasks using labeld data, aprocess also known as supervised fine-tuning, further improving it performance.

In the following sections of thsi lab, you will explore pretraining objectives, loading pretrained models, data preparation, and the fine-tuning process.  By the end, you will have a solid understanding of pretraining and self-supervised fine-tuning, empowering you to apply these techniques to solve real-world NLP problems.

Let's start with loading a pretrained model from Hugging Face and making an inference:

In [ ]:
model = AutoModelForCausalLM.from_pretrained("facebook/opt-350m")
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-350m")

pipe = pipeline("text-generation", model=model,tokenizer=tokenizer)
print(pipe("This movie was really")[0]["generated_text"])



config.json:   0%|          | 0.00/644 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/663M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/662M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/441 [00:00<?, ?B/s]

Device set to use cuda:0


This movie was really really really really really really really really really really really really really really really really


## Pre-training Objectives

Pre-training objectives are crucial components of the pre-training process for transforers.  These objectives define the tasks that model is trained on during the pre-training phse, allowing it to learn meaningful contextual representations of language.  Three commonly used pre-training objectives are masked language modeling (MLM), next sentence prediction (NSP) and next Token prediction.

1. Masked Language Modeling (MLM):  Masked language modeling involves randomly masking some words in a sentence and training the model to predict the masked words based on teh context provided by the surrounding words (i.e., words that apppear either before or after the masked word).  The objectives is to enable the model to learn contextual understanding and fill in missing information.

2. Next Sentence Prediction (NSP): Next sentence prediction invloves training the model to predict whether tow sentences are consecutive in theoriginal text or randomly chosen from the corpus.  This objective helps the model learn sentence-level relationships and understand the coherence between sentences.

3. Next Toke Prediction: In this objective, the model is trained to predict the next token in a sequence of text.  The model is presented with a sequence of text and must learn to predict the most likely next token based on the context.

It is important to note that different pre-trainednmodels may use variations or combinations of these objectives, depending on the specific architecture and training setup.

# Self-supervised training of a BERT model

Training a BERT (Bidirectional Encoder Representations from Transformers) model is a complex and time-consuming process that requires a large corpus of unlabeled text data and significant computational resources.  However,we provide you with a simplified exercixe to demonstrate the steps involved inpre-training a BERT model using the Masked Langugae Modeling (MLM) objective.

For this exercise, we will use the Hugging Face Transfomers library, which provides pre-implemented BERT models and tools for pre-training.  You will be instructed to:
prepare the train dataset
Train a Tokenize
Preprocess the dataset
Pre-train BERT using an MLM task
Evaluate the trained model



## Importing required datasets

The WikiTexr dataset is a widely used benchmark dataset in the field of natural language processing (NLP).  The dataset contains a large amount of text extracted from Wikipedia, which is a vast online encyclopedia covering a wide range of topics.  The articles in the WikiText dataset are preprocessed to remove formatting, hyperlinks, and other metadata, resulting in aclean text corpus.

The Wikitext dataset hs 4 different configs, and is divided into three parts: a training set, a validation set, and a test set.  The training set is used for training language models, while the validation and test sets are used for evaluating the performance of the models.  First, let us load the datasets and concatenante them together to crete a big dataset

In [ ]:
# Load the datasets
!pip install datasets
import datasets
from datasets import load_dataset
dataset = load_dataset("wikitext", "wikitext-2-raw-v1")
# dataset = load_dataset("wikitext", "wikitext-103-raw-v1")

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/733k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/6.36M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/657k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

In [ ]:
print(dataset)

DatasetDict({
    test: Dataset({
        features: ['text'],
        num_rows: 4358
    })
    train: Dataset({
        features: ['text'],
        num_rows: 36718
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 3760
    })
})


In [ ]:
# Check a sample record
dataset["train"][400]

{'text': " When Mason was injured in warm @-@ ups late in the year , Columbus was without an active goaltender on their roster . To remedy the situation , the team signed former University of Michigan goaltender Shawn Hunwick to a one @-@ day , amateur tryout contract . After being eliminated from the NCAA Tournament just days prior , Hunwick skipped an astronomy class and drove his worn down 2003 Ford Ranger to Columbus to make the game . He served as the back @-@ up to Allen York during the game , and the following day , he signed a contract for the remainder of the year . With Mason returning from injury , Hunwick was third on the team 's depth chart when an injury to York allowed Hunwick to remain as the back @-@ up for the final two games of the year . In the final game of the season , the Blue Jackets were leading the Islanders 7 – 3 with 2 : 33 remaining when , at the behest of his teammates , Head Coach Todd Richards put Hunwick in to finish the game . He did not face a shot . 

In [ ]:
dataset["train"] = dataset["train"].select([i for i in range(1000)])
dataset["test"] = dataset["test"].select([i for i in range(200)])

Below files are next used in creating TextDataset objects for the training:

In [ ]:
# Path to save the datasets to text files
output_file_train = "wikitext_dataset_train.txt"
output_file_test = "wikitext_dataset_test.txt"

# Open theoutput file in write mode
with open(output_file_train, "w", encoding="utf-8") as f:
  # Iterate over each example in the dataset
  # Use dataset['train'] to access the 'train' split instead of dataset("train")
  for example in dataset['train']:
    # Write the example text to the file
    f.write(example["text"] + "\n")

# Open the output file in write mode
with open(output_file_test, "w", encoding="utf-8") as f:
  # Iterate over ach example in the dataset
  # Use dataset['test'] to access the 'test' split instead of dataset("test")
  for example in dataset['test']:
    # Write the example text to the file
    f.write(example["text"] + "\n")

You need to define a tokenizer to be used for tokenizing the dataset.

In [ ]:
# Create a tokenizer from exisitng one to re-use special tokens
bert_tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
model_name = 'bert-base-uncased'

model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name, is_decoder=True)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`


## Training a Tokenizer (Optional)

in the previous cell, you created an instance of tokenizer from a pre-trained BERT tokenizer.  If you want to train the tokenizer on your own dataset, you can uncomment code below.  This is specially helpful when using transformers for specific areas such as medicine where token are somehow different than the general tokens that tokenizers are crated based on.  (You can skip this step if you do not want to train the tokenizer on your specific data):

In [ ]:
## Create a python generator to dynamically load the data
def batch_iterator(batch_size=10000):
  for i in tqdm(range(0, len(dataset), batch_size)):
    yield dataset['train'][i : i + batch_size]["text"]

## Create a tokenizer from existing one to re-use special tokens
bert_tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

## Train the tokenizer using our won dataset
bert_tokenizer = bert_tokenizer.train_new_from_iterator(text_iterator=batch_iterator(), vocab_size=30522)

  0%|          | 0/1 [00:00<?, ?it/s]

## Pretraining

In this step, we defind the configuration of the BERT model and create the model:

### Define the BERT Configuration

Here, we define the configuration settings for a BERT model using BertConfig.  This includes setting various parameters related to the model's architecture:

vocab_size=30522: Specifies the size of the vocabulary.  This number should match the vocabulary size used by the tokenizer.
hidden_size=768:  Sets the size of the hidden layers.
num_hidden_layers=12:  Determines the number of hidden layers in the transformer model.
num_attention_heads=12:  Sets the number of attention heads in each attention layer.
intermediate_size=3072:  Specifies the size of the "intermediate" (i.e., feed-forward) layer within the transformer.

In [ ]:
# Define the BERT configuration
config = BertConfig(
   vocab_size=len(bert_tokenizer.get_vocab()), # Specify tje vocabulary size
   hidden_size=768, # Set the hidden size
   num_hidden_layers=12,  # Set the number of layers
   num_attention_heads=12,  # Set the number of attention heads
   intermediate_size=3072,  # Set the intermediate size
)

Create the BERT model for pre-training:

In [ ]:
# Create the BERTmodel for pre-training
model = BertForMaskedLM(config)

Check model configuration

In [ ]:
# Check model configuration
model

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(12577, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwi

## Define the Traing Dataset

Here, we define a training dataset using the TextDataset class, which is suited for loading and processing text data for training language models. This setup typically involves a few key parameters:

- **tokenizer=bert_tokenizer**: Specifies the tokenizer to be used. Here, `bert_tokenizer` is an instance of a BERT tokenizer, responsible for converting text into tokens that the model can understand.
- **file_path="wikitext_dataset_train.txt"**: The path to the pre-training data file. This should point to a text file containing the training data.
- **block_size=128**: Sets the desired block size for training. This defines the length of the sequences that the model will be trained on

The `TextDataset` class is designed to take large pieces of text (such as those found in the specified file), tokenize them, and efficiently handle them in manageable blocks of the specified size.

In [ ]:
# Prepare the pre-training data as a TextDataset
train_dataset = TextDataset(
    tokenizer=bert_tokenizer,
    file_path="wikitext_dataset_train.txt",  # Path to your pre-training data file
    block_size=128  # Set the desired block size for training
)
test_dataset = TextDataset(
    tokenizer=bert_tokenizer,
    file_path="wikitext_dataset_test.txt",  # Path to your pre-training data file
    block_size=128  # Set the desired block size for training
)

Token indices sequence length is longer than the specified maximum sequence length for this model (55468 > 512). Running this sequence through the model will result in indexing errors


examining one sample the token indexes are shown here with the block size.

In [ ]:
train_dataset[0]

tensor([    2,    31,   619,   799,  1414,    31,  3541,   593,   619,    22,
           29, 12315,   799,    12,  3188,    29,   105,   103,  5411,    15,
          490,    17,   619,   184,   171,  2553,    22,    13,    15,  3533,
         3583,   191,   216,   619,   799,  1414,  1540,  2033,    15,   256,
           35,  5440,  1124,    32,    16,    32,  1843,  2189,   398,  1762,
          251,  3403,   188,  3159,    17,  2434,   209,   171,  2290,  4152,
           17,  1349,   180,  1685,  1213,   180,  2033,    15,   288,   256,
          171,  1310,   398,   180,   171,   619,  1220,    17,  6800,   171,
          859,  7535,   184,  5440,   188,  1151,    32,    16,    32,   533,
         1972,   216,   408,  4371,    15,   171,  1504, 10331,  9722,   191,
          171,   447,   398,   188,  6352,   171,     6,  1864,     6,    15,
           35,  3527,  1062,  2489,  4112,   171,  1807,   184,  3574,   514,
          171,   753,  7208,   527,   407,   852,  2471,     3])

Then, we prepare data for the MLM task (masking random tokens):
### Define the Data Collator for Language Modeling
This line of code sets up a `DataCollatorForLanguageModeling` from the Hugging Face Transformers library. A data collator is used during training to dynamically create batches of data. For language modeling, particularly for models like BERT that use masked language modeling (MLM), this collator prepares training batches by automatically masking tokens according to a specified probability. Here are the details of the parameters used:

- **tokenizer=bert_tokenizer**: Specifies the tokenizer to be used with the data collator. The `bert_tokenizer` is responsible for tokenizing the text and converting it to the format expected by the model.
- **mlm=True**: Indicates that the data collator should mask tokens for masked language modeling training. This parameter being set to `True` configures the collator to randomly mask some of the tokens in the input data, which the model will then attempt to predict.
- **mlm_probability=0.15**: Sets the probability with which tokens will be masked. A probability of 0.15 means that, on average, 15% of the tokens in any sequence will be replaced with a mask token.

In [ ]:
# Prepare the data collator for language modeling
data_collator = DataCollatorForLanguageModeling(
    tokenizer=bert_tokenizer, mlm=True, mlm_probability=0.15
)

In [ ]:
# check how collator transforms a sample input data record
data_collator([train_dataset[0]])

{'input_ids': tensor([[    2,    31,   619,   799,  1414,    31,  3541,   593,   619,    22,
             29, 12315,   799,    12,  3188,    29,   105,     4,     4,    15,
            490,    17,   619,   184,   171,  2553,    22,    13,    15,     4,
           3583,   191,   216,   619,   799,  1414,  1540,  2033,    15,   256,
             35,  5440,  1124,    32,    16,    32,  1843,  2189,   398,  1762,
            251,  3403,   188,  3159,    17,  2434,   209,   171,  2290,  4152,
             17,  1349,     4,  1685,  1213,     4,  2033,    15,   288,   256,
            171,  1310,   398,   180,   171,   619,  1220,    17,  6800,   171,
            859,  7535,   184,  5440,     4,  1151,    32,    16,    32,   533,
           1972,   216,   408,  4371,    15,   171,  1504,     4,  9722,   191,
              4,   447,   398,   188,  6352,   171,     6,  1864,     4,    15,
             35,  3527,  1062,  2489,  4112,   171,  1807,   184,  3574,     4,
            171,   753,  72

Now, we train the BERT Model using the Trainer module. (For a complete list of training arguments, check [here](https://huggingface.co/docs/transformers/v4.33.2/en/main_classes/trainer#transformers.TrainingArguments)):
This section configures the training process by specifying various parameters that control how the model is trained, evaluated, and saved:

- **output_dir="./trained_model"**: Specifies the directory where the trained model and other output files will be saved.
- **overwrite_output_dir=True**: If set to `True`, this will overwrite the contents of the output directory if it already exists. This is useful when running experiments multiple times.
- **do_eval=True**: Enables evaluation of the model. If `True`, the model will be evaluated at the specified intervals.
- **evaluation_strategy="epoch"**: Defines when the model should be evaluated. Setting this to "epoch" means the model will be evaluated at the end of each epoch.
- **learning_rate=5e-5**: Sets the learning rate for training the model. This is a typical learning rate for fine-tuning BERT-like models.
- **num_train_epochs=10**: Specifies the number of training epochs. Each epoch involves a full pass over the training data.
- **per_device_train_batch_size=2**: Sets the batch size for training on each device. This should be set based on the memory capacity of your hardware.
- **save_total_limit=2**: Limits the total number of model checkpoints to be saved. Only the most recent two checkpoints will be kept.
- **logging_steps=20**: Determines how often to log training information, which can help monitor the training process.

In [ ]:
!pip install transformers==4.33.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.9/119.9 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 122.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 156.8 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.1
    Uninstalling tokenizers-0.21.1:
      Successfully uninstalled tokenizers-0.21.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.52.0.dev0
    Uninstalling transformers-4.52.0.dev0:
      Successfully uninstalled transformers-4.52.0.dev0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 3.4.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.33.2 which is incompatible.


In [ ]:
# Define the training arguments
training_args = TrainingArguments(
    output_dir="./trained_model",  # Specify the output directory for the trained model
    overwrite_output_dir=True,
    do_eval=True,
    # evaluation_strategy="epoch",  # Remove or comment out this line
    learning_rate=5e-5,
    num_train_epochs=10,  # Specify the number of training epochs
    per_device_train_batch_size=2,  # Set the batch size for training
    save_total_limit=2,  # Limit the total number of saved checkpoints
    logging_steps = 20
)

# Instantiate the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# Start the pre-training
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: petkao (petkao-peter-kao-associates) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
20,8.709400
40,7.779700
60,7.692100
80,7.568100
100,7.268500
120,7.192900
140,7.189600
160,7.169800
180,6.961500
200,7.281600


TrainOutput(global_step=2200, training_loss=6.867123385342684, metrics={'train_runtime': 243.3395, 'train_samples_per_second': 18.082, 'train_steps_per_second': 9.041, 'total_flos': 289464654336000.0, 'train_loss': 6.867123385342684, 'epoch': 10.0})

# Evaluateing Model Performance

Let's check the performance of the trained model. Perplexity is commonly used to compare different language models or different configurations of the same model.
After training, perplexity can be calculated on a held-out evaluation dataset to assess the model's performance. The perplexity is calculated by feeding the evaluation dataset through the model and comparing the predicted probabilities of the target tokens with the actual token values that are masked.

A lower perplexity score indicates that the model has a better understanding of the language and is more effective at predicting the masked tokens. It suggests that the model has learned useful representations and can generalize well to unseen data.

In [ ]:
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 4006.00


# Loading the saved model

If you want to skip training and load the model that you trained for 10 epochs, go ahead and uncomment the following cell:

In [ ]:
!wget 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/BeXRxFT2EyQAmBHvxVaMYQ/bert-scratch-model.pt'
# Load the saved state dictionary using the correct vocabulary size
config = BertConfig(
   vocab_size=30522, # Specify tje vocabulary size
   hidden_size=768, # Set the hidden size
   num_hidden_layers=12,  # Set the number of layers
   num_attention_heads=12,  # Set the number of attention heads
   intermediate_size=3072,  # Set the intermediate size
)
# Create the BERTmodel for pre-training
model = BertForMaskedLM(config)
model.load_state_dict(torch.load('bert-scratch-model.pt', map_location=torch.device('cpu')))

--2025-05-11 06:15:06--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/BeXRxFT2EyQAmBHvxVaMYQ/bert-scratch-model.pt
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 438141816 (418M) [binary/octet-stream]
Saving to: ‘bert-scratch-model.pt’

bert-scratch-model. 100%[===================>] 417.84M  22.1MB/s    in 20s     

2025-05-11 06:15:26 (21.3 MB/s) - ‘bert-scratch-model.pt’ saved [438141816/438141816]



<All keys matched successfully>

The simplest way to try out the model for inference is to use it in a pipeline(). Instantiate a pipeline for fill-mask with your model, and pass your text to it. If you like, you can use the top_k parameter to specify how many predictions to return:

In [ ]:
# Define the input text with a masked token
text = "This is a [MASK] movie!"

# Create a pipeline for the "fill-mask" task
mask_filler = pipeline("fill-mask", model=model,tokenizer=bert_tokenizer)

# Generate predictions by filling the mask in the input text
results = mask_filler(text) #top_k parameter can be set

# Print the predicted sequences
for result in results:
    print(f"Predicted token: {result['token_str']}, Confidence: {result['score']:.2f}")

Device set to use cuda:0


Predicted token: ##ular, Confidence: 0.05
Predicted token: -, Confidence: 0.04
Predicted token: /, Confidence: 0.04
Predicted token: 200, Confidence: 0.04
Predicted token: h, Confidence: 0.02


You can see that [MASK] is replaced by the most frequent token. This weak performance can be due to insufficient training, lack of training data, model architecture, or not tuning hyperparameters. Let's try a pretrained model from Hugging Face:

# Inferencing a pretrained BERT model

In [ ]:
# Load the pretrained BERT model and tokenizer
pretrained_model = BertForMaskedLM.from_pretrained('bert-base-uncased')
pretrained_tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

# Define the input text with a masked token
text = "This is a [MASK] movie!"

# Create the pipeline
mask_filler = pipeline(task='fill-mask', model=pretrained_model,tokenizer=pretrained_tokenizer)

# Perform inference using the pipeline
results = mask_filler(text)
for result in results:
    print(f"Predicted token: {result['token_str']}, Confidence: {result['score']:.2f}")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0


Predicted token: great, Confidence: 0.16
Predicted token: horror, Confidence: 0.08
Predicted token: good, Confidence: 0.08
Predicted token: bad, Confidence: 0.05
Predicted token: fantastic, Confidence: 0.04


This pretrianed model performs way better than the model you just trained for a few epochs using a single dataset. Still, pretrained models cannot be used for specific tasks, such as sentiment extraction or sequence classification. This is why supervised fine-tuning methods are introduced.